In [ ]:
!pip install web3

In [ ]:
import time
import json
import requests
import pandas as pd
from tqdm import tqdm
from web3 import Web3
from web3.exceptions import ABIFunctionNotFound

In [ ]:
ETH_API_KEY = "NATDJD6I31EU3VE55UAD1FBY3VHGF8EE27"
ETH_URL = "https://api.etherscan.io/api"

RPC_URL = "https://rpc.ankr.com/eth/b1bdd2819d3da7be4760d270af7ef985a74725cbe337c348bca633d096806c5a"
w3 = Web3(Web3.HTTPProvider(RPC_URL))

getAccountBalance_url = "https://rpc.ankr.com/multichain/79258ce7f7ee046decc3b5292a24eb4bf7c910d7e39b691384c7ce0cfb839a01/?ankr_getAccountBalance="

In [ ]:
factories = {
    "uniswapv2": {
        "address" : ["0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f"],
        "pool" : []
        },
}

In [ ]:
def get_abi(address):
    params = {
        "module": "contract",
        "action": "getabi",
        "address": address,
        "apikey": ETH_API_KEY
    }
    response = requests.post(ETH_URL, params=params)
    time.sleep(0.2)
    data = json.loads(response.text)
    result = data['result']
    return result

In [ ]:
def get_contract(address):
    abi = get_abi(address)
    checksum_address = Web3.to_checksum_address(address)
    contract = w3.eth.contract(address=checksum_address, abi=abi)
    return contract

In [ ]:
def get_fee_from_fee(contract, name):
    deno_dict = {
        "pancakeswap": 10000*100,
        "curve": 10**8,
        "uniswapv3": 10000*100
    }
    if name in deno_dict:
        fee = float(contract.functions.fee().call())
        fee = fee / deno_dict[name]
        return fee
    elif name == "fraxswap":
        fee = 10000 - float(contract.functions.fee().call())
        fee = fee / 10000
        return fee
    return None


# NomiswapStable
def get_fee_from_swapFee(contract, name):
    try:
        swapFee = float(contract.functions.swapFee().call())
        max_fee = float(contract.functions.MAX_FEE().call())
        fee = swapFee / max_fee
        return fee
    except Exception as e:
        return None


def get_fee_from_getSwapFeePercentage(contract, name):
    if name == "balancerv2":
        swapFeePercentage = float(contract.functions.getSwapFeePercentage().call())
        fee = swapFeePercentage / 10**18
        return fee
    return None


# saddle
def get_fee_from_swapStorage(contract, name):
    if name == "saddle":
        swapStorage = contract.functions.swapStorage().call()
        swapFee = float(swapStorage[4])
        fee = swapFee / 10**8
        return fee
    return None


def get_fee_from_tradingFeePercent(contract, name):
    if name == "linkswap":
        tradingFeePercent = float(contract.functions.tradingFeePercent().call())
        fee = tradingFeePercent / 10**6
        return fee
    return None


def get_fee_from_factory(contract, name):
    if name == "crodefiswap":
        factory_address = contract.functions.factory().call()
        factory_source = get_source_code(factory_address)
        factory_contract = get_contract(factory_address, factory_source)
        totalFeeBasisPoint = float(factory_contract.functions.totalFeeBasisPoint().call())
        fee = totalFeeBasisPoint / 10000
        return fee
    return None


def get_fee(contract, name):
    fix_fee_dict = {
        "uniswapv2": 3/1000,
        "sushiswap": 3/1000,
        "verseexchange": 3/1000,
        "saitaswap": 2/1000,
        "sakeswap_pool": 3/1000,
        "xchange": 200/100000,
        "kingswap": 25/10/1000,
    }

    fee_fun = [
        get_fee_from_fee,
        get_fee_from_swapFee,
        get_fee_from_getSwapFeePercentage,
        get_fee_from_swapStorage,
        get_fee_from_tradingFeePercent,
        get_fee_from_factory
    ]

    if name in fix_fee_dict:
        return fix_fee_dict[name]

    for fun in fee_fun:
        fee = fun(contract, name)
        if fee:
            break
    return fee

In [ ]:
def get_tokens(contract):
    try:
        token0 = contract.functions.token0().call().lower()
        token1 = contract.functions.token1().call().lower()
        return token0, token1
    except Exception:
        return None, None

In [ ]:
def get_pool_from_allPairs(factory, start, end):
    rows = []
    name = factory
    for factory_address in factories[factory]["address"]:
        factory_contract = get_contract(factory_address)
        try:
            allPairsLength = factory_contract.functions.allPairsLength().call()
            if end > allPairsLength:
                end = allPairsLength
            for i in tqdm(range(start, end)):
                try:
                    pool_address = factory_contract.functions.allPairs(i).call().lower()
                    contract = get_contract(pool_address)
                    fee = get_fee(contract, name)
                    token0, token1 = get_tokens(contract)
                    row = {
                        "name" : name,
                        "factory" : factory_address,
                        "fee" : fee,
                        "contract" : pool_address,
                        "token0" : token0,
                        "token1" : token1,
                    }
                    rows.append(row)
                    time.sleep(0.2)
                except ValueError:
                    continue
        except ABIFunctionNotFound:
            continue
    pool = pd.DataFrame(rows)
    pool = pool.drop_duplicates(ignore_index = True)
    return pool

In [ ]:
def get_tvl(pool):
    token_type = ['NATIVE']
    rows = []

    for i in tqdm(range(0, len(pool))):
        payload = {
            "jsonrpc": "2.0",
            "method": "ankr_getAccountBalance",
            "params": {
                "blockchain": ["eth"],
                "walletAddress": pool.contract[i],
                "nativeFirst": True,
                "onlyWhitelisted": True,
                "pageSize": 2
            },
            "id": 1
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json"
        }

        response = requests.post(getAccountBalance_url, json=payload, headers=headers)
        time.sleep(1)
        if response.status_code == 200:
            try:
                data = json.loads(response.text)
                tvl = data['result']['totalBalanceUsd']
            except KeyError:
                tvl = None
            row = {
                "name" : pool.name[i],
                "factory" : pool.factory[i],
                "tvl" : tvl,
                "fee" : pool.fee[i],
                "contract" : pool.contract[i],
                "token0" : pool.token0[i],
                "token1" : pool.token1[i],
            }

            rows.append(row)
    pool = pd.DataFrame(rows)
    pool = pool.dropna(axis=0).reset_index(drop=True)
    return pool

In [ ]:
for count in range(0, 10000):
    start = count * 1000
    end = start + 1000
    pool = get_pool_from_allPairs("uniswapv2", start, end)
    if len(pool) <= 0:
        break
    file_name = f"part{count}.csv"
    pool.to_csv(file_name, index=False)

    pool = get_tvl(pool)
    file_name = f"part{count}.csv"
    pool.to_csv(file_name, index=False)

100%|███████████████████████████████████████| 1000/1000 [20:45<00:00,  1.25s/it]


100%|███████████████████████████████████████| 1000/1000 [20:14<00:00,  1.21s/it]


100%|█████████████████████████████████████████| 465/465 [10:29<00:00,  1.35s/it]
0it [00:00, ?it/s]


In [ ]:
files = [f"part{i}.csv" for i in range(count)]

In [ ]:
combined_df = pd.DataFrame()

In [ ]:
for file in files:
    df = pd.read_csv(file)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

combined_df = combined_df.drop_duplicates()
combined_df

,name,factory,tvl,fee,contract,token0,token1
0,uniswapv2,0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f,5.734411e+07,0.003,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
1,uniswapv2,0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f,1.303778e+01,0.003,0x3139ffc91b99aa94da8a2dc13f1fc36f9bdc98ee,0x8e870d67f660d95d5be530380d0ec0bd388289e1,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
2,uniswapv2,0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f,9.590303e+02,0.003,0x12ede161c702d1494612d19f05992f43aa6a26fb,0x06af07097c9eeb7fd685c692751d5c66db49c215,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
3,uniswapv2,0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f,1.106897e+07,0.003,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x6b175474e89094c44da98b954eedeac495271d0f,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
4,uniswapv2,0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f,1.656952e+02,0.003,0x07f068ca326a469fc1d87d85d448990c8cba7df9,0x408e41876cccdc0f92210600ef50372656052a38,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
...,...,...,...,...,...,...,...
279272,uniswapv2,0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f,1.891746e+03,0.003,0x65464f8df2c04f29c825d22c1b2331180a821d5f,0x96cd34cc8472ba86ff52d16055460bc0c1e12a58,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
279273,uniswapv2,0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f,5.132139e+03,0.003,0xae01ffe5b76796c007ca0db5950f259c5bd370b2,0x760a4ca65fc9fd57082f51492f059ee0bbafd83e,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
279274,uniswapv2,0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f,1.873190e+03,0.003,0x13856d64d503203958e7defb9498b422fef7e656,0x2b7e74c85adc1c2ab9321b228b3b24d1dbc8a52c,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
279275,uniswapv2,0x5c69bee701ef814a2b6a3edd4b1652cb9cc5aa6f,0.000000e+00,0.003,0x45c973a29c8ba569ba75201f8d13ee6277c0d496,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xc358990ce78c5629ec18f277416ab9417f60a5ad


In [ ]:
combined_df.to_csv('eth_uniswapv2.csv', index=False)